# Foreign function interfaces

- foreign function interfaces (FFI) are used to interoperate with non-Rust code.
- `extern` is the main FFI mechanism
- FFI can also be used to call another Rust binary

## Crossing Boundaries with extern

### Symbols

- names assigned to particular addresses in a given segment of your binary that allow you to share memory
- compiler generates unique names
- names can’t be random, for FFI purposes
- during compilation, the _linker_ replaces symbols with:
  - actual memory addresses (static linking)
  - code that resolves the reference at runtime (dynamic linking)
- `extern` declare a symbol as defined outside the compilation unit (resolved during linking)
- `#[no_mangle]` attribute forces to keep the name of the symbol (to expose Rust code outside)
- note: static variables are immutable by default (unregarding they are into a `extern` block)
- dynamic linking allow easy upgrades and produces smaller binaries, but forces to distribute the right version of the libraries

### Calling Conventions

- specify how to call a foreign function:
  - the structure of the stack frame
  - how to pass arguments
  - how to return
  - how to restore the CPU context
- Specified with the `extern` argument
- Unlike C, the Rust ABI is not standard (don’t use it to integrate with other languages)
- Unwinding generally works only with regular Rust functions
- A function’s calling convention is part of its type
- Normal Rust code is hard to be used as a compiled library (because of generics)
  - produce C-compatible a library instead

## Types Across Language Boundaries

- type layout is crucial and must be consistent across the boundary

## Type Matching

- type information is erased upon compilation
- use the same convention on both sides
- Rust library contains an implementation of C types
- more complex types may need manual mapping
  - for structs, pay attention to layout and padding
  - C-like bitsets don’t work out-of-the-box

## Allocations

- pointer can be passed through boundaries, but memory must be deallocated by the same allocator
  - interfaces may expose freeing functions
  - or function may expect the caller to provide allocated memory (better options, but only when size is known in advance)

### Callbacks

- `extern` functions can be passed as function pointers to other `extern` functions
  - e.g.: `extern "C" fn(c_int) -> c_int`
- use `std::panic::catch_unwind` to detect the panic in any `extern` function

### Safety

- most of the code that interacts with the FFI will be unsafe
- to write safe code, manually enforce the invariants in the safe wrapper

#### References and Lifetimes

- declare as `&mut` a pointer that may mutate outside the boundary
- the safe wrapper should declare proper lifetimes

#### Send and Sync

- mark types as `Send` and `Sync` only when documented as thread-safe

#### Pointer Confusion

- to mask internal representation, the interface can return an _opaque pointer_ (i.e.: `void *`)
- Rust allow defining zero-sized types such as empty structs to wrap them
  - will increse the type safety

## bindgen and Build Scripts

- the utility `bindgen` generate Rust bindings from a C header file
  - they are platform-specific
  - can be integrated into the build script
  - `bindgen` works well with `C` but doesn’t support every `C++` feature
  - `cbindgen` is the inverse tool that generates `C` bindings for your Rust library
